In [1]:
!pip install pycolbertdb

  Using cached datamodel_code_generator-0.25.6-py3-none-any.whl.metadata (23 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached argcomplete-3.3.0-py3-none-any.whl.metadata (16 kB)
  Using cached genson-1.3.0-py3-none-any.whl.metadata (28 kB)
  Using cached inflect-5.6.2-py3-none-any.whl.metadata (21 kB)
Using cached datamodel_code_generator-0.25.6-py3-none-any.whl (108 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached argcomplete-3.3.0-py3-none-any.whl (42 kB)
Using cached genson-1.3.0-py3-none-any.whl (21 kB)
Using cached inflect-5.6.2-py3-none-any.whl (33 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0


In [2]:
!pip install llama-index

In [3]:
!pip install llama-index-readers-web

In [4]:
!pip install datasets

In [97]:
import os
from dotenv import load_dotenv
from pycolbertdb.client import Colbertdb
from datasets import load_dataset
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI
import random


load_dotenv()

True

In [ ]:
dataset = load_dataset("microsoft/ms_marco", split="validation", name="v2.1")

In [36]:
only_answers = [x for x in dataset if x["answers"] != ['No Answer Present.']]

In [63]:

sample = random.sample(list(only_answers), 1000)

In [73]:
passages = []
for item in sample:
    passages.extend(item["passages"]["passage_text"])

['Gambit is actually the second character from that film to be rebooted within the franchise, the first being Deadpool. Interestingly, Ryan Reynolds played both versions of the merc with a mouth. Before Tatum hits the big screen as Gambit, there will be three other X-Men films coming to theaters.',
 "Fox also unveiled release date for Fantastic Four 2, even though the first movie hasn't come out yet We also got a date for Fantastic Four 2 X-Men Spinoff Gambit Finally Has a Release Date",
 'Channing Tatum will play the X-Men character Gambit. Created by Chris Claremont and Jim Lee in 1990, the New Orleans native mutant quickly became a fan favorite due to his kinetic energy manipulation abilities and penchant for stealing.',
 'Alita: Battle Angel - release date pushed back to Dec. The Predator - release date pushed back one month; Death on the Nile - release date; Foster - release date; Wildlife - studio; Official Secrets - Keira Knightley and Matt Smith to star in the real-life spy thr

10003


In [100]:
url = os.getenv("COLBERTDB_URL")
api_key = os.getenv("COLBERTDB_API_KEY")
store_name = os.getenv("COLBERTDB_STORE_NAME")
openai_key = os.getenv("OPENAI_API_KEY")
client = Colbertdb(url, api_key, store_name)
openai_client = OpenAI(api_key=openai_key, model="gpt-4-turbo")
print(openai_key)

sk-proj-SO92lxW96iOMkQLdxKeUT3BlbkFJu1j9Bcl2n9IvHQs3fXGt


In [88]:
documents = []
for passage in passages:
    documents.append({"content": passage, "metadata": {}})

client.create_collection(name="msmarco", documents=documents, options={"force_create": True})
collection = client.load_collection("msmarco")

In [101]:

qa_prompt_tmpl_str = """\
---------------------
Passages: {passages}
---------------------

---------------------
Question: {question}
---------------------

---------------------
Answer: {answer}
---------------------

Score the group of passages (as a whole) on its relevance to answering the question given the provided answer.

Output: Score from 1 - 10 for each passage.
"""
for entry in sample[0:2]:
    content = ''
    question = entry['query']
    answer = entry['answers'][0]
    response = collection.search(question, k=3)
    for document in response.documents:
        content += document.content + '\n\n'
    prompt_template= PromptTemplate(qa_prompt_tmpl_str)
    prompt = prompt_template.format(passages=content, question=question, answer=answer)
    print(prompt)
    # openai_client.complete(prompt)

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['passages', 'question', 'answer'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='---------------------\nPassages: {passages}\n---------------------\n\n---------------------\nQuestion: {question}\n---------------------\n\n---------------------\nAnswer: {answer}\n---------------------\n\nScore the group of passages (as a whole) on its relevance to answering the question given the provided answer.\n\nOutput: Score from 1 - 10 for each passage.\n'
metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['passages', 'question', 'answer'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='---------------------\nPassages: {passages}\n---------------------\n\n---------------------\nQuestion: {question}\n---------------------\n\n---------------------\nAnswer: {answer}\n---------------------\n\nScore the group of passages (as a whole